In [39]:
from selenium import webdriver
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests
from bs4 import BeautifulSoup


In [40]:
def web_scrape(url):
    options = Options()
    options.add_argument("--enable-performance-logging")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    log = driver.get_log("performance")
    driver.quit()
    
    return log

In [41]:
url = "https://www.oddsportal.com/football/belgium/jupiler-pro-league/"
logs = web_scrape(url)

In [42]:
with open("oddsportal_network_dump.json", 'w', encoding="utf-8") as f:
    f.write("[")
    for log in logs:
        network_log = json.loads(log["message"])["message"]
        if "Network.response" in network_log["method"] or "Network.request" in network_log["method"] or "Network.webSocket" in network_log["method"]:
            # if("")
            f.write(json.dumps(network_log)+",")
    f.write("{}]")

In [43]:
with open("oddsportal_network_dump.json", "r", encoding="utf8") as f:
    logs = json.loads(f.read())

urls = []
for log in logs:
    try:
        if(log["params"]["type"] == 'XHR'):
            url = log["params"]["request"]["url"]
            urls.append(url)
    except:
        pass

filter_list = [".svg", ".png", "/js/", ".js", ".css", ".ico"]
filtered_urls = [url for url in urls if not any(filter_str in url for filter_str in filter_list)]

for url in filtered_urls:
    print(url)


https://content.livesportmedia.eu/delivery?zones=3991,3989,3990,5989,6061&clientType=PC&no_flash=1&programmatic=false
https://geolocation.onetrust.com/cookieconsentpub/v1/geo/location
https://www.oddsportal.com/ajax-sport-country-tournament_/1/zun4qzgk/X0/1/?_=1714565427
https://www.oddsportal.com/ajax-getCount/MyBookmarks/
https://www.oddsportal.com/ajax-getCount/MyCoupon/
https://www.oddsportal.com/set-time-zone/37/


# Finding the request(s) needed

In [63]:
def check_url_contents(url, words=None, regex_patterns=None):
    headers = { 
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    
    results = {'words': {}, 'regex_patterns': {}}
    
    if response.status_code == 200:
        content = response.text
        
        if words:
            for word in words:
                count = content.lower().count(word.lower())
                results['words'][word] = count
    
    return results

In [72]:
def rank_urls(urls, words=None, regex_patterns=None):
    url_scores = {}
    for url in urls:
        restults = check_url_contents(url, words,regex_patterns)
        
        score=0
        if words:
            score+= sum(restults['words'].values())
        if regex_patterns:
            score+= sum(restults['regex_patterns'].values())
        url_scores[url] = score
    
    sorted_urls = sorted(url_scores.items(), key=lambda x:x[1], reverse=True)

    #returns a list sorted based on the amount of occurences of both the strings and the regex
    return sorted_urls
        

In [77]:
result = rank_urls(filtered_urls, ['odd', 'bet'])

for entry in result:
    print(entry[0])

https://www.oddsportal.com/ajax-sport-country-tournament_/1/zun4qzgk/X0/1/?_=1714565427
https://content.livesportmedia.eu/delivery?zones=3991,3989,3990,5989,6061&clientType=PC&no_flash=1&programmatic=false
https://geolocation.onetrust.com/cookieconsentpub/v1/geo/location
https://www.oddsportal.com/ajax-getCount/MyBookmarks/
https://www.oddsportal.com/ajax-getCount/MyCoupon/
https://www.oddsportal.com/set-time-zone/37/
